In [1]:
from util import *

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
for site in tqdm(["OhaweBeach", "Waitara"]):
    df = gpd.read_file(f"Shapefiles/{site}_intersects.shp")
    df.crs = 2193
    df = enrich_df(df)
    azimuth_lookup = get_azimuth_dict(f"Shapefiles/{site}_TransectLines.shp")
    for model in tqdm(SUPPORTED_MODELS):
        results = predict(df, azimuth_lookup, model)
        polygon = prediction_results_to_polygon(results)
        output_shapefile = f"Projected_Shoreline_Polygons/{site}_{model}.shp"
        polygon.to_file(output_shapefile, driver="ESRI Shapefile")
        print(f"Wrote {output_shapefile}")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/856 [00:00<?, ?it/s]

Wrote Projected_Shoreline_Polygons/OhaweBeach_linear.shp


  0%|          | 0/856 [00:00<?, ?it/s]

...

<ipython-input-2-51d15bc30443>:7: RankWarning: Polyfit may be poorly conditioned
  results = predict(df, azimuth_lookup, model)


Wrote Projected_Shoreline_Polygons/Waitara_BH.shp


  0%|          | 0/910 [00:00<?, ?it/s]

<ipython-input-2-51d15bc30443>:7: RankWarning: Polyfit may be poorly conditioned
  results = predict(df, azimuth_lookup, model)


Wrote Projected_Shoreline_Polygons/Waitara_Sunamura.shp
